<a href="https://colab.research.google.com/github/AFNANAMIN/-hierarchical-federated-learning/blob/master/SQL_queries_for_Facebook_%26_AdWords_performance_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

def get_worksheet_data(sheet_url, worksheet_index):
    # Define the scope of the credentials
    scope = ["https://www.googleapis.com/auth/drive"]

    # Authenticate with the Google Sheets API using the credentials file
    creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
    client = gspread.authorize(creds)

    # Open the Google Sheets document by URL
    sheet = client.open_by_url(sheet_url)

    # Get the worksheet by index
    worksheet = sheet.get_worksheet(worksheet_index)

    # Get all the data from the worksheet as a list of lists
    data = worksheet.get_all_values()

    # Convert the data to a Pandas DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])

    return df

In [ ]:
facebook_ads=get_worksheet_data("https://docs.google.com/spreadsheets/d/16Kilc52_44_iH_X3jqVv6yV_-ETYO7BgKWi6V05qnaM/edit#gid=265449672",2)

In [ ]:
google_ads=get_worksheet_data("https://docs.google.com/spreadsheets/d/16Kilc52_44_iH_X3jqVv6yV_-ETYO7BgKWi6V05qnaM/edit#gid=265449672",1)

In [ ]:
google_ads.columns

Index(['day', 'campaign', 'clicks', 'conversions', 'cost', 'convrate', 'ctr',
       'avgcpc'],
      dtype='object')

In [ ]:
facebook_ads.columns

Index(['impressions', 'campaign_id', 'account_name', 'campaign_name', 'ctr',
       'clicks', 'adset_id', 'ad_id', 'cpc', 'cpm', 'adset_name', 'spend',
       'date_start', 'conversions', 'platform'],
      dtype='object')

In [ ]:
google_ads

,day,campaign,clicks,conversions,cost,convrate,ctr,avgcpc
0,2020-07-29 0:00:00,Shopping-Generic-USA-MAT-2-MP [pros],10,0,127730000,0,0.16,12773000
1,2020-07-29 0:00:00,Shopping-Generic-USA-MAT-3-LP [pros],8,0,27880000,0,0.1,3485000
2,2020-07-29 0:00:00,PS-Generic-Competitor-Birch Living [pros],3,0,23910000,0,2.07,7970000
3,2020-07-29 0:00:00,PS-Brand-Sample-Compare To [brand],4,0,34310000,0,23.53,8577500
4,2020-07-29 0:00:00,PS-Generic-Competitor-Avocado [pros],23,0,199680000,0,1.78,8681739
...,...,...,...,...,...,...,...,...
995,2020-06-19 0:00:00,PS-Generic-Chemical Free [ATC] [pros],2,0,4910000,0,3.12,2455000
996,2020-06-19 0:00:00,PS-Brand-Sample-Product Sizes [brand],6,0,46110000,0,30,7685000
997,2020-06-19 0:00:00,PS-Generic-Organic Hybrid Product [ATC] [pros],0,0,0,0,0,0
998,2020-06-19 0:00:00,DAC-Generic ZZZ[pros],0,0,0,0,0,0


In [ ]:
import re

# Define a function to extract the platform from the campaign name
def extract_platform(campaign):
    if re.search(r'\[rt\]', campaign, re.IGNORECASE):
        return 'RT'
    elif re.search(r'\[pros\]', campaign, re.IGNORECASE):
        return 'PROS'
    else:
        return 'Other'

# Apply the function to create the 'platform' column
google_ads['platform'] = google_ads['campaign'].apply(extract_platform)


In [ ]:
import sqlite3
conn = sqlite3.connect(":memory:")
google_ads.to_sql("google_ads", conn, index=False)


1000

In [ ]:
query = """
    SELECT day,
           SUM(clicks) as total_clicks,
           SUM(conversions) as total_conversations,
           SUM(cost) as total_cost ,
           AVG(convrate) as avg_conrate,
           AVG(ctr) as avg_ctr,
           AVG(avgcpc) as avgcpc,
           SUM(clicks) OVER (PARTITION BY platform ORDER BY day ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING) AS clicks_previous_day,
           SUM(clicks) OVER (PARTITION BY platform ORDER BY day ROWS BETWEEN 7 PRECEDING AND 7 PRECEDING) AS clicks_7_days_ago
    FROM google_ads
    GROUP BY day, platform
"""
result_df = pd.read_sql_query(query,conn)

# Google Ads Performance Metrics


In [ ]:
import plotly.express as px

fig = px.line(result_df, x='day', y=['total_clicks', 'total_conversations', 'total_cost', 'avg_conrate', 'avg_ctr', 'avgcpc', 'clicks_previous_day', 'clicks_7_days_ago'], title='Google Ads Performance Metrics')
fig.show()

In [ ]:
facebook_ads['platform'] = facebook_ads['campaign_name'].apply(extract_platform)

In [ ]:
facebook_ads.to_sql("facebook_ads", conn, index=False)

1000

In [ ]:
query = """
    SELECT 
    platform,
    date_start ,
    campaign_name,
    SUM(clicks) AS clicks,
    SUM(spend) AS cost,
    AVG(ctr) AS ctr,
    AVG(cpc) AS avgcpc,
    AVG(cpm) AS avgcpm,
    SUM(impressions) AS impressions,
    SUM(conversions) AS conversions,
    AVG(conversions/clicks) AS convrate,
    SUM(CASE WHEN date_start = DATE(date_start, '-1 day') THEN clicks ELSE 0 END) AS clicks_previous_day,
    SUM(CASE WHEN date_start = DATE(date_start, '-7 day') THEN clicks ELSE 0 END) AS clicks_7_days_ago
FROM 
    facebook_ads
GROUP BY 
    platform, date_start, campaign_name;
"""
result_df = pd.read_sql_query(query, conn)

In [ ]:
result_df.columns

Index(['platform', 'date_start', 'campaign_name', 'clicks', 'cost', 'ctr',
       'avgcpc', 'avgcpm', 'impressions', 'conversions', 'convrate',
       'clicks_previous_day', 'clicks_7_days_ago'],
      dtype='object')

# Facebook Ads Performance Metrics

In [ ]:
fig = px.line(result_df, x='date_start', y='clicks', color='platform', title='Clicks by Platform')
fig.show()

In [ ]:
fig = px.line(result_df, x='date_start', y='cost', color='platform', 
              title='Cost by Platform')
fig.show()

fig = px.line(result_df, x='date_start', y='ctr', color='platform', 
              title='CTR by Platform')
fig.show()